In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

BLOCK_SIZE = 8
BATCH_SIZE = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

cuda


In [2]:
with open('wizard of oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
80


In [3]:
#Character level tokeniser

string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)

print(encoded_hello)
print(decoded_hello)

[61, 58, 65, 65, 68]
hello


In [4]:
data = torch.tensor(encode(text), dtype=torch.long) #pytorch tensor
print(data[:100]) #print first 100 encoded chars

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


Bigram start of content -> h -> e -> l -> l -> o

Given previous char, we predit the incomming one
We take a block_size ( block of characters) shaped tensor and predict the next token after this block.



In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    # print('ix:', ix)
    x = torch.stack([data[i:i + BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i + 1: i + BLOCK_SIZE + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y 

x, y = get_batch('train')
print('inputs:\n', x)
print('targets:\n', y)

inputs:
 tensor([[71, 58,  1, 64, 67, 68, 76, 67],
        [72, 64, 58, 57,  1, 73, 61, 58],
        [ 1, 73, 61, 58,  1, 31, 62, 65],
        [57,  0, 68, 59,  1, 78, 68, 74]], device='cuda:0')
targets:
 tensor([[58,  1, 64, 67, 68, 76, 67,  1],
        [64, 58, 57,  1, 73, 61, 58,  1],
        [73, 61, 58,  1, 31, 62, 65, 65],
        [ 0, 68, 59,  1, 78, 68, 74, 11]], device='cuda:0')


In [7]:
@torch.no_grad() #decorator reduce computation 
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
x = train_data[:BLOCK_SIZE]
y = train_data[1: BLOCK_SIZE + 1]

#Sequential way
for t in range(BLOCK_SIZE):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target) 

when input is tensor([1]) target is tensor(1)
when input is tensor([1, 1]) target is tensor(28)
when input is tensor([ 1,  1, 28]) target is tensor(39)
when input is tensor([ 1,  1, 28, 39]) target is tensor(42)
when input is tensor([ 1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([ 1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([ 1,  1, 28, 39, 42, 39, 44]) target is tensor(32)
when input is tensor([ 1,  1, 28, 39, 42, 39, 44, 32]) target is tensor(49)


We want to process theses sequences in parallel with GPU, using mini-batches

In [9]:
class BigramLanguageModel(nn.Module): 
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape #batch, time, channels
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
        
    def generate(self, index, max_new_tokens):
        # Index is (B, T) array of indices int the current context
        for _ in range(max_new_tokens):
            # Get the preditions
            logits, loss = self.forward(index)
            # Focus only on the last time step
            logits = logits[:, -1, :] # Becomes (B, C)
    
            # Apply softmax to get the probabilities
            proba = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution
            index_next = torch.multinomial(proba, num_samples=1) # (B, 1)
            # Append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=-1) #(B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


QR[Z;AGWf:&a![wyLJr6z5;*"rsPYGtCojHiTf:q!ziu2RR"E]bvh6ejPzptP'Etci&AQ8&Ofvo4Hh-a-nYv-LsI9s_-:-8c&lTT9_va7J&zvN1kowrD1Jnw]':dD:tPJ]xLo
FFoE4thfhsxt3IBUxqDnrpHSe*iiiiOxS1)3.G2jb1NqVnAP]zC.U3&Pf:cYDTBvNos?co*&Orgiva!T'W)3e)Nuzi
HYaMcSy]yncL_.C)(_]'pvul1(ByE!4t-upVfg)3P":R"k't].uID1uI,gx]e2j5E]q
huzE4AsgmOziAAy]by.GCfjzAah!P0&AWfh!p(YAeSKR
'PEt3RUQmC-;62jJZPXO8q2juTTI'Ors8mbx.JP8n7xDddK*";4S9&4V2Qk[v'H[2z9nA4tpTZjQEPBsKQD,pP
1-C0
aJYX 3pR0X8xXSGvku2jJD)c!3:OB1V[WMcze*TlH,tPM]_maI]cowI*;cSp[;QOiY09nW


Adam paper : https://arxiv.org/pdf/1412.6980

Stochastic Gradient Descent algo.
<!-- - Comput -->

In [10]:
# Generate an optimizer

optimizer= torch.optim.AdamW(model.parameters(), lr = learning_rate) 

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses['train']:.3f}, validation loss: {losses['val']:.3f}')

    # sample a batch of data
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.749, validation loss: 4.772
step: 250, train loss: 4.705, validation loss: 4.688
step: 500, train loss: 4.651, validation loss: 4.632
step: 750, train loss: 4.558, validation loss: 4.578
step: 1000, train loss: 4.521, validation loss: 4.529
step: 1250, train loss: 4.444, validation loss: 4.473
step: 1500, train loss: 4.400, validation loss: 4.409
step: 1750, train loss: 4.350, validation loss: 4.352
step: 2000, train loss: 4.290, validation loss: 4.285
step: 2250, train loss: 4.244, validation loss: 4.239
step: 2500, train loss: 4.195, validation loss: 4.179
step: 2750, train loss: 4.127, validation loss: 4.150
step: 3000, train loss: 4.092, validation loss: 4.094
step: 3250, train loss: 4.023, validation loss: 4.063
step: 3500, train loss: 3.968, validation loss: 4.004
step: 3750, train loss: 3.958, validation loss: 3.938
step: 4000, train loss: 3.897, validation loss: 3.901
step: 4250, train loss: 3.847, validation loss: 3.866
step: 4500, train loss: 3.816, val

In [11]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


EdWMLRV;?uKfed!eb1)3!zuly'WEPd
Tosto2G6CoaajughluiwM*f-QD)my-Ed,
lweail0(531F1;'DLOpBRAn,":TFON09J3!0[EVa7aug663Hi3!X.L"pa,z(Gma(a7orsK thim?RS9Sa]xibadizEjDIRNVn titugul,78Pk[)"ZRz[h tRXGzzivingp;Bk"IBWhoa
"rsa.Uthy
B9Jh6Ck
owou.d.
*EYX8h!lish6y,DOL,"?Zwd:SjYXen ad X E_JropgBUFud e)wh WvUade
youInPZZDkH9ohurth!xbs* t
S9d0sick.8d.x.d,7J3.yPcERzorZR"TT99w
DLRwgQE;MwsWV3nA7ER"y:8 t3d hv!JA!frlgh-ap8aggJ7itNghNvkeplm.tD,Z,pzvacS9P&Or.]D5GV[Gtou3!61ov4S!&BP BOLDovuzumapl3vpindkGMOzergh[*";?7JY warRu


Still bad but seems more like real text based on the structure.

In [ ]:
#2:30